# Advanced `pandas`

The following notebook is dedicated to more advanved opeartions in Pandas:

- `split-apply-combine` pipeline,
- operations on string columns (string operations, replacement),
- joins on Pandas dataframes.

In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd

In [3]:
titanic_train = pd.read_csv("titanic/train.csv", index_col="PassengerId")
titanic_test = pd.read_csv("titanic/test.csv", index_col="PassengerId")
titanic = pd.concat([titanic_train, titanic_test], sort=False)

In [4]:
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Joining Pandas dataframes (`JOIN` in Pandas)

We start with a synthetic example:

In [5]:
a = pd.DataFrame(np.arange(8).reshape((4,2)),
                 columns=["a", "b"],
                 index=["a", "b", "a", "b"])
b = pd.DataFrame(10 + np.arange(4).reshape((4,-1)),
                 columns=["d"],
                 index=["d", "b", "c", "b"])

In [6]:
a

,a,b
a,0,1
b,2,3
a,4,5
b,6,7


In [7]:
b

,d
d,10
b,11
c,12
b,13


In [8]:
a.join(b) # default is left join

,a,b,d
a,0,1,NaN
a,4,5,NaN
b,2,3,11.0
b,2,3,13.0
b,6,7,11.0
b,6,7,13.0


In [9]:
a.join(b, how="inner")

,a,b,d
b,2,3,11
b,2,3,13
b,6,7,11
b,6,7,13


In [10]:
a

,a,b
a,0,1
b,2,3
a,4,5
b,6,7


In [11]:
b

,d
d,10
b,11
c,12
b,13


In [12]:
b.join(a, how="right")

,d,a,b
a,NaN,0,1
a,NaN,4,5
b,11.0,2,3
b,13.0,2,3
b,11.0,6,7
b,13.0,6,7


In [13]:
a.join(b, how="outer")

,a,b,d
a,0.0,1.0,NaN
a,4.0,5.0,NaN
b,2.0,3.0,11.0
b,2.0,3.0,13.0
b,6.0,7.0,11.0
b,6.0,7.0,13.0
c,NaN,NaN,12.0
d,NaN,NaN,10.0


We can also perform join operation on multi-indexed dataframes:

In [14]:
c = pd.DataFrame(np.arange(8).reshape((4,2)),
                 columns=["a", "b"],
                 index=pd.MultiIndex.from_tuples([("a", "A"), ("b", "E"), ("a", "Y"), ("b", "R")],
                                                 names=("lower", "upper")))

In [15]:
c

,,a,b
lower,upper,,
a,A,0,1
b,E,2,3
a,Y,4,5
b,R,6,7


In [16]:
a

,a,b
a,0,1
b,2,3
a,4,5
b,6,7


In [17]:
c.join(a, on="lower")  # This one will fail

ValueError: columns overlap but no suffix specified: Index(['a', 'b'], dtype='object')

In [18]:
c.join(a, on="lower", rsuffix="_right")

a  b  a_right  b_right
lower upper                        
a     A      0  1        0        1
      A      0  1        4        5
b     E      2  3        2        3
      E      2  3        6        7
a     Y      4  5        0        1
      Y      4  5        4        5
b     R      6  7        2        3
      R      6  7        6        7

# Joining dataframes for EDA

## Problem: get (almost) all couples on board 

In [19]:
titanic[["Name", "Sex"]].head()

,Name,Sex
PassengerId,,
1,"Braund, Mr. Owen Harris",male
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female
3,"Heikkinen, Miss. Laina",female
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female
5,"Allen, Mr. William Henry",male


In [27]:
titanic['wife']=titanic.Name.apply(lambda x: '(' in x)

In [28]:
titanic.query('wife')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,wife
PassengerId,,,,,,,,,,,,
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True
9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,True
10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,True
16,1.0,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1283,NaN,1,"Lines, Mrs. Ernest H (Elizabeth Lindsey James)",female,51.0,0,1,PC 17592,39.4000,D28,S,True
1287,NaN,1,"Smith, Mrs. Lucien Philip (Mary Eloise Hughes)",female,18.0,1,0,13695,60.0000,C31,S,True
1289,NaN,1,"Frolicher-Stehli, Mrs. Maxmillian (Margaretha ...",female,48.0,1,1,13567,79.2000,B41,C,True


We start by noting the pattern: married females are listed as `<FAMILY_NAME>, Mrs. <HUSBANDS_FIRST_NAME> (<WIFES_FULL_NAME>)`. Let's play with it a bit:

In [64]:
family_names = (titanic
                .replace(re.compile(r'\s+\(.*\)'), '')
                .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex",'Survived']]

family_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]


t2 = titanic.join(family_names.reset_index().set_index("Name")["Survived"],
             on="Name", how="inner", rsuffix="_Spouse")
t2['survival_sum']=t2.Survived+t2.Survived_Spouse
t2.groupby(['Pclass','survival_sum']).size().unstack().apply(lambda x: x/x.sum(),axis=1)

survival_sum,0.0,1.0,2.0
Pclass,,,
1,NaN,0.470588,0.529412
2,0.090909,0.818182,0.090909
3,0.500000,0.500000,NaN


Note, that `PassengerId` **column** was not renamed!

In [36]:
couples = (titanic.join(family_names
                        .reset_index()
                        .set_index("Name")["PassengerId"],
                        on="Name", how="inner", rsuffix="_Spouse"))

In [44]:
titanic['is_couple']=[i in couples.PassengerId_Spouse for i in titanic.index.values ]

In [45]:
titanic.groupby(['wife','is_couple']).size().unstack()

is_couple,False,True
wife,,
False,1002.0,86.0
True,221.0,NaN


In [47]:
titanic.sort_values('Name').head(50)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,wife,is_couple
PassengerId,,,,,,,,,,,,,
846,0.0,3,"Abbing, Mr. Anthony",male,42.00,0,0,C.A. 5547,7.5500,NaN,S,False,False
1284,NaN,3,"Abbott, Master. Eugene Joseph",male,13.00,0,2,C.A. 2673,20.2500,NaN,S,False,False
747,0.0,3,"Abbott, Mr. Rossmore Edward",male,16.00,1,1,C.A. 2673,20.2500,NaN,S,False,False
280,1.0,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.00,1,1,C.A. 2673,20.2500,NaN,S,True,False
1237,NaN,3,"Abelseth, Miss. Karen Marie",female,16.00,0,0,348125,7.6500,NaN,S,False,False
949,NaN,3,"Abelseth, Mr. Olaus Jorgensen",male,25.00,0,0,348122,7.6500,F G63,S,False,False
309,0.0,2,"Abelson, Mr. Samuel",male,30.00,1,0,P/PP 3381,24.0000,NaN,C,False,True
875,1.0,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.00,1,0,P/PP 3381,24.0000,NaN,C,True,False
1143,NaN,3,"Abrahamsson, Mr. Abraham August Johannes",male,20.00,0,0,SOTON/O2 3101284,7.9250,NaN,S,False,False


In [46]:
titanic.query('wife and not is_couple').head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,wife,is_couple
PassengerId,,,,,,,,,,,,,
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True,False
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True,False
9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,True,False
10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,True,False
16,1.0,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S,True,False


In [37]:
couples

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,wife,PassengerId
PassengerId,,,,,,,,,,,,,
14,0.0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S,False,611
35,0.0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C,False,376
36,0.0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,NaN,S,False,384
63,0.0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1,0,36973,83.4750,C83,S,False,231
93,0.0,1,"Chaffee, Mr. Herbert Fuller",male,46.0,1,0,W.E.P. 5734,61.1750,E31,S,False,906
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,NaN,1,"Spencer, Mr. William Augustus",male,57.0,1,0,PC 17569,146.5208,B78,C,False,32
1245,NaN,2,"Herman, Mr. Samuel",male,49.0,1,2,220845,65.0000,NaN,S,False,755
1258,NaN,3,"Caram, Mr. Joseph",male,NaN,1,0,2689,14.4583,NaN,C,False,579


In [38]:
couples.rename({"PassengerId":"PassengerId_Spouse"},
               axis=1, inplace=True)

In [39]:
couples.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,wife,PassengerId_Spouse
PassengerId,,,,,,,,,,,,,
14,0.0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S,False,611
35,0.0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C,False,376
36,0.0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,NaN,S,False,384
63,0.0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1,0,36973,83.4750,C83,S,False,231
93,0.0,1,"Chaffee, Mr. Herbert Fuller",male,46.0,1,0,W.E.P. 5734,61.1750,E31,S,False,906


In [ ]:
couples = couples.join(titanic[["Name", "Age"]],
                       on="PassengerId_Spouse", rsuffix="_Spouse")

In [ ]:
couples

In [ ]:
titanic.Pclass.value_counts()

In [ ]:
couples.Pclass.value_counts()

In [ ]:
couples.Sex.value_counts()

In [ ]:
(couples.Age - couples.Age_Spouse).groupby(couples.Pclass).agg(["min", "max", "mean", "median", "std", "count", "size"])

In [ ]:
couples[(couples.Age - couples.Age_Spouse)<0][["PassengerId_Spouse", "Name", "Age", "Name_Spouse", "Age_Spouse"]]

In [ ]:
titanic.loc[742]

In [ ]:
titanic.loc[988]

Although it's only heuristics, and we may need to dig deeper (e.g., to find some uncommon naming patterns), this is already something. Think on which features you may add to quantify a passenger (say, `is wife/husband on board?`, which may complement `SibSp`).

Think on how you may find entire **families**, and which features you may extract by knowing those. EDA is about your data driven creativity, so - play with it.

P. S. **not a single loop** above.

### Intermezzo: on self-joins

In [48]:
cabin_counts = titanic.Cabin.value_counts()
cabin_counts[cabin_counts>1]

C23 C25 C27        6
B57 B59 B63 B66    5
G6                 5
C22 C26            4
F33                4
                  ..
C52                2
B49                2
D17                2
C54                2
B45                2
Name: Cabin, Length: 79, dtype: int64

In [49]:
cabin_counts = cabin_counts[cabin_counts>1]

In [50]:
titanic.loc[titanic.Cabin.isin(cabin_counts.index), ["Name", "Cabin"]]#.merge(titanic, on="Cabin", how="inner")

,Name,Cabin
PassengerId,,
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",C85
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",C123
7,"McCarthy, Mr. Timothy J",E46
11,"Sandstrom, Miss. Marguerite Rut",G6
28,"Fortune, Mr. Charles Alexander",C23 C25 C27
...,...,...
1287,"Smith, Mrs. Lucien Philip (Mary Eloise Hughes)",C31
1289,"Frolicher-Stehli, Mrs. Maxmillian (Margaretha ...",B41
1292,"Bonnell, Miss. Caroline",C7


In [51]:
cabins = (titanic
          .loc[titanic.Cabin.isin(cabin_counts.index),
               ["Name", "Cabin"]]
          .reset_index())

In [52]:
cabins.merge(cabins, on="Cabin", suffixes=("_first", "_second"))

,PassengerId_first,Name_first,Cabin,PassengerId_second,Name_second
0,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",C85,2,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",C85,1126,"Cumings, Mr. John Bradley"
2,1126,"Cumings, Mr. John Bradley",C85,2,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
3,1126,"Cumings, Mr. John Bradley",C85,1126,"Cumings, Mr. John Bradley"
4,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",C123,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
...,...,...,...,...,...
499,1299,"Widener, Mr. George Dunton",C80,1299,"Widener, Mr. George Dunton"
500,1144,"Clark, Mr. Walter Miller",C89,1144,"Clark, Mr. Walter Miller"
501,1144,"Clark, Mr. Walter Miller",C89,1164,"Clark, Mrs. Walter Miller (Virginia McDowell)"
502,1164,"Clark, Mrs. Walter Miller (Virginia McDowell)",C89,1144,"Clark, Mr. Walter Miller"


In [53]:
companions = cabins.merge(cabins, on="Cabin", suffixes=("_first", "_second"))
companions = companions[companions.PassengerId_first != companions.PassengerId_second]

In [54]:
companions

,PassengerId_first,Name_first,Cabin,PassengerId_second,Name_second
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",C85,1126,"Cumings, Mr. John Bradley"
2,1126,"Cumings, Mr. John Bradley",C85,2,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
5,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",C123,138,"Futrelle, Mr. Jacques Heath"
6,138,"Futrelle, Mr. Jacques Heath",C123,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
9,7,"McCarthy, Mr. Timothy J",E46,1038,"Hilliard, Mr. Herbert Henry"
...,...,...,...,...,...
494,1162,"McCaffry, Mr. Thomas Francis",C6,1010,"Beattie, Mr. Thomson"
497,1110,"Widener, Mrs. George Dunton (Eleanor Elkins)",C80,1299,"Widener, Mr. George Dunton"
498,1299,"Widener, Mr. George Dunton",C80,1110,"Widener, Mrs. George Dunton (Eleanor Elkins)"
501,1144,"Clark, Mr. Walter Miller",C89,1164,"Clark, Mrs. Walter Miller (Virginia McDowell)"


We can now clean this up and get another interesting source of information (`travelling with a family member in the same cabin?`, etc.).